In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
df = pd.read_csv("Why are employees leaving.csv")  #Loading file into my Notebook
df.head()

,ID,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Division,salary
0,1,0.38,0.53,2,157,3,0,1,0,sales,low
1,2,0.80,0.86,5,262,6,0,1,0,sales,medium
2,3,0.11,0.88,7,272,4,0,1,0,sales,medium
3,4,0.72,0.87,5,223,5,0,1,0,sales,low
4,5,0.37,0.52,2,159,3,0,1,0,sales,low


In [2]:
# Calculate entropy based on the 'left' column
import math

def calculate_entropy(data, target_column):
    """
    Calculate entropy for a given target column
    Entropy = -Σ(p * log2(p)) where p is the probability of each class
    """
    # Get value counts and probabilities
    value_counts = data[target_column].value_counts()
    total_samples = len(data)
    
    entropy = 0
    # print(f"Distribution of '{target_column}' column:")
    # print(f"Total samples: {total_samples}")
    
    for value, count in value_counts.items():
        probability = count / total_samples
        if probability > 0:  # Avoid log(0)
            entropy -= probability * math.log2(probability)
        # print(f"  {target_column}={value}: {count} samples ({probability:.4f} probability)")
    
    return entropy

# Calculate entropy for the 'left' column
# entropy_left = calculate_entropy(df, 'left')
# print(f"\n Baseline Entropy of the dataset based on 'left' column: {entropy_left:.4f}")



In [3]:
# INFORMATION GAIN CALCULATION FOR DISCRETE FEATURES (Example for a discrete feature)

def calculate_information_gain_discrete(data, feature_column, target_column):
    # Step 1: Calculate original entropy (baseline entropy of target variable)
    # print(f"=== INFORMATION GAIN ANALYSIS FOR '{feature_column.upper()}' COLUMN ===\n")
    
    original_entropy = calculate_entropy(data, target_column)
    # print(f"\nBASELINE ENTROPY of '{target_column}' column: {original_entropy:.4f}")
    
    # Step 2: Get unique values in the feature column to split on
    unique_values = data[feature_column].unique()
    # print(f"\nUNIQUE VALUES in '{feature_column}' column: {unique_values}")
    
    total_samples = len(data)
    weighted_entropy = 0
    
    for value in unique_values:
        # Create subset of data for this feature value
        subset = data[data[feature_column] == value]
        subset_size = len(subset)
        
        # Calculate entropy for this subset
        subset_entropy = calculate_entropy(subset, target_column)
        
        # Calculate weight (proportion of total data this subset represents)
        weight = subset_size / total_samples
        
        # Add to weighted entropy calculation
        weighted_entropy += weight * subset_entropy
    
    # Step 4: Calculate Information Gain
    information_gain = original_entropy - weighted_entropy
   
    # print(f"\nInformation Gain is {information_gain:.4f} for {feature_column} column")
    return {'information_gain': round(information_gain, 4),
            'weighted_entropy': round(weighted_entropy, 4), 
            'original_entropy': round(original_entropy, 4),
            'feature_column': feature_column,
            'split': None}

In [4]:
calculate_information_gain_discrete(df, 'salary', 'left')

{'information_gain': 0.0209,
 'weighted_entropy': 0.7709,
 'original_entropy': 0.7918,
 'feature_column': 'salary',
 'split': None}

In [5]:
# Information Gain for Continuous Features

def calculate_information_gain_continuous(data, feature_column, target_column):
    # Check if data is empty
    if len(data) == 0:
        print(f"Warning: No data available for feature '{feature_column}'")
        return {'information_gain': 0, 
                'weighted_entropy': 0, 
                'original_entropy': 0, 
                'feature_column': feature_column, 
                'split': None}
    
    # Sort the unique values of the feature column
    original_entropy = calculate_entropy(data, target_column)
    # print(f"\nBASELINE ENTROPY of '{target_column}' column: {original_entropy:.4f}")
    total_samples = len(data)

    # print(f"{feature_column}")
    
    sorted_values = np.sort(data[feature_column].unique())
    # print("Sorted values:", sorted_values, len(sorted_values))
    
    # Check if there are no unique values or only one unique value
    if len(sorted_values) < 1:
        print(f"Warning: Cannot create splits for feature '{feature_column}' - insufficient unique values")
        return {'information_gain': 0, 
                'weighted_entropy': original_entropy, 
                'original_entropy': round(original_entropy, 4), 
                'feature_column': feature_column, 
                'split': sorted_values[0] if len(sorted_values) == 1 else None}


    def create_feature_splits(sorted_values):
        modified_sortedValues = []
        for i in range(len(sorted_values) - 1):
            modified_sortedValues.append((sorted_values[i] + sorted_values[i+1])/2)
        return modified_sortedValues
    
    feature_splits = create_feature_splits(sorted_values)
    # print("Feature splits:", feature_splits, len(feature_splits))
    
    # Check if feature_splits is empty
    if len(feature_splits) == 0:
        print(f"Warning: No splits possible for feature '{feature_column}'")
        return {'information_gain': 0, 
                'weighted_entropy': original_entropy, 
                'original_entropy': round(original_entropy, 4), 
                'feature_column': feature_column, 
                'split': None}
    
    selected_split = feature_splits[0]
    max_information_gain = 0
    max_weighted_entropy = original_entropy
    
    # Calculate the information gain for each range
    for i,_ in enumerate(feature_splits):
        # print(f"Split {i}: {feature_splits[i], sorted_values[i], sorted_values[i+1]}")
        subset1 = data[data[feature_column] >= feature_splits[i]]
        subset1_size = len(subset1)
        subset1_entropy = calculate_entropy(subset1, target_column)
        weight1 = subset1_size / total_samples
        # print("Subset:", len(subset))
        
        subset2 = data[data[feature_column] < feature_splits[i]]
        subset2_size = len(subset2)
        subset2_entropy = calculate_entropy(subset2, target_column)
        weight2 = subset2_size / total_samples

        # print(f"Split {i}: {feature_splits[i]}")
        # print(f"Subset1: {subset1_size} samples, entropy: {subset1_entropy:.4f}")
        # print(f"Subset2: {subset2_size} samples, entropy: {subset2_entropy:.4f}")

        weighted_entropy = weight1 * subset1_entropy + weight2 * subset2_entropy
        information_gain = original_entropy - weighted_entropy

        if information_gain > max_information_gain:
            max_information_gain = information_gain
            selected_split = feature_splits[i]
            max_weighted_entropy = weighted_entropy
        # print("Subset1:", len(subset1))
    # print(f"\nMax information gain is {max_information_gain:.4f} for {feature_column} column and split is {selected_split}, {max_weighted_entropy}")
    return {'information_gain': round(max_information_gain, 4), 
            'weighted_entropy': round(max_weighted_entropy, 4), 
            'original_entropy': round(original_entropy, 4), 
            'feature_column': feature_column, 
            'split': round(selected_split, 3) if selected_split is not None else None}

# calculate_information_gain_continuous(df, 'satisfaction_level', 'left')


In [6]:
calculate_information_gain_continuous(df, 'satisfaction_level', 'left')

{'information_gain': 0.1926,
 'weighted_entropy': 0.5993,
 'original_entropy': 0.7918,
 'feature_column': 'satisfaction_level',
 'split': 0.465}

In [7]:
# df.describe()
def calculate_max_information_gain(df, df_name):
    satisfaction= calculate_information_gain_continuous(df, 'satisfaction_level', 'left')
    last_evaluation= calculate_information_gain_continuous(df, 'last_evaluation', 'left')
    salary= calculate_information_gain_discrete(df, 'salary', 'left')
    division= calculate_information_gain_discrete(df, 'Division', 'left')
    average_montly_hours= calculate_information_gain_continuous(df, 'average_montly_hours', 'left')
    time_spend_company= calculate_information_gain_continuous(df, 'time_spend_company', 'left')
    number_project= calculate_information_gain_continuous(df, 'number_project', 'left')
    work_accident= calculate_information_gain_discrete(df, 'Work_accident', 'left')
    promotion_last_5years= calculate_information_gain_discrete(df, 'promotion_last_5years', 'left')
    
    features = [satisfaction, last_evaluation, salary, division, average_montly_hours, time_spend_company, number_project, work_accident, promotion_last_5years]

    information_gains = [feature['information_gain'] for feature in features]
    max_information_gain = max(information_gains)
    print(f"\nOptimal Feature information for {df_name} \n {features[information_gains.index(max_information_gain)]}")
    return features[information_gains.index(max_information_gain)]

In [8]:
#STEP 1 => ROOT NODE. Data set is the whole data frame. (DEPTH 0)
calculate_max_information_gain(df, "Root Node")

# print(f"{calculate_information_gain_continuous(df, 'satisfaction_level', 'left')[0]}")


Optimal Feature information for Root Node 
 {'information_gain': 0.1926, 'weighted_entropy': 0.5993, 'original_entropy': 0.7918, 'feature_column': 'satisfaction_level', 'split': 0.465}


{'information_gain': 0.1926,
 'weighted_entropy': 0.5993,
 'original_entropy': 0.7918,
 'feature_column': 'satisfaction_level',
 'split': 0.465}

In [9]:
#STEP 2 => Split tree based on Satisfaction Level into 2 and repeat the process for each split. (DEPTH 1)

#NODE 1 => Sat. Level <= 0.465
df1 = df[df['satisfaction_level'] <= 0.465]
print(f"\nNODE 1 => Sat. Level <= 0.465")
calculate_max_information_gain(df1, "NODE 1")

#NODE 2 => Sat. Level > 0.465
df2 = df[df['satisfaction_level'] > 0.465]
print(f"\nNODE 2 => Sat. Level > 0.465")
calculate_max_information_gain(df2, "NODE 2")




NODE 1 => Sat. Level <= 0.465

Optimal Feature information for NODE 1 
 {'information_gain': 0.1802, 'weighted_entropy': 0.7877, 'original_entropy': 0.9679, 'feature_column': 'satisfaction_level', 'split': 0.115}

NODE 2 => Sat. Level > 0.465

Optimal Feature information for NODE 2 
 {'information_gain': 0.185, 'weighted_entropy': 0.2716, 'original_entropy': 0.4567, 'feature_column': 'time_spend_company', 'split': 4.5}


{'information_gain': 0.185,
 'weighted_entropy': 0.2716,
 'original_entropy': 0.4567,
 'feature_column': 'time_spend_company',
 'split': 4.5}

In [10]:
#STEP 2 => Split NODE 1 and NODE 2 into further branches (DEPTH 2)
#ROOT NODE 

#NODE 1 => Sat. Level <= 0.465
print(f"\nNODE 1 => Sat. Level <= 0.465")

#NODE 1.1 => Sat. Level <= 0.115
df11 = df1[df1['satisfaction_level'] <= 0.115]
print(f"\nNODE 1.1 => Sat. Level <= 0.115")
calculate_max_information_gain(df11, "NODE 1.1")

#NODE 1.2 => Sat. Level > 0.115
df12 = df1[df1['satisfaction_level'] > 0.115]
print(f"\nNODE 1.2 => Sat. Level > 0.115")
calculate_max_information_gain(df12, "NODE 1.2")

#----------------------------------

#NODE 2 => Sat. Level > 0.465
print(f"\nNODE 2 => Sat. Level > 0.465")
#NODE 2.1 => time spend <= 4.5
df21 = df2[df2['time_spend_company'] <= 4.5]
print(f"\nNODE 2.1 => time spend <= 4.5")
calculate_max_information_gain(df21, "NODE 2.1")

#NODE 2.2 => time spend > 4.5
df22 = df2[df2['time_spend_company'] > 4.5]
print(f"\nNODE 2.2 => time spend > 4.5")
calculate_max_information_gain(df22, "NODE 2.2")



NODE 1 => Sat. Level <= 0.465

NODE 1.1 => Sat. Level <= 0.115

Optimal Feature information for NODE 1.1 
 {'information_gain': 0, 'weighted_entropy': 0.0, 'original_entropy': 0.0, 'feature_column': 'satisfaction_level', 'split': 0.095}

NODE 1.2 => Sat. Level > 0.115

Optimal Feature information for NODE 1.2 
 {'information_gain': 0.5379, 'weighted_entropy': 0.4621, 'original_entropy': 1.0, 'feature_column': 'number_project', 'split': 2.5}

NODE 2 => Sat. Level > 0.465

NODE 2.1 => time spend <= 4.5

Optimal Feature information for NODE 2.1 
 {'information_gain': 0.0063, 'weighted_entropy': 0.103, 'original_entropy': 0.1093, 'feature_column': 'average_montly_hours', 'split': 290.5}

NODE 2.2 => time spend > 4.5

Optimal Feature information for NODE 2.2 
 {'information_gain': 0.3694, 'weighted_entropy': 0.626, 'original_entropy': 0.9954, 'feature_column': 'last_evaluation', 'split': 0.805}


{'information_gain': 0.3694,
 'weighted_entropy': 0.626,
 'original_entropy': 0.9954,
 'feature_column': 'last_evaluation',
 'split': 0.805}

In [11]:
#STEP 3 
#Focusing on NODE 1.2 (DEPTH 3)

#NODE 1 => Sat. Level <= 0.465
print(f"\nNODE 1 => Sat. Level <= 0.465")

#NODE 1.1 => Sat. Level <= 0.115
print(f"\nNODE 1.1 => Sat. Level <= 0.115")

#NODE 1.2 => Sat. Level > 0.115
print(f"\nNODE 1.2 => Sat. Level > 0.115")

#NODE 1.2.1 => Projects <= 2.5
print(f"\nNODE 1.2.1 => Projects <= 2.5")
df121 = df12[df12['number_project'] <= 2.5]
print(f"Size of df121: {len(df121)}")
calculate_max_information_gain(df121, "NODE 1.2.1")

#NODE 1.2.2 => Projects > 2.5
print(f"\nNODE 1.2.2 => Projects > 2.5")
df122 = df12[df12['number_project'] > 2.5]
calculate_max_information_gain(df122, "NODE 1.2.2")




NODE 1 => Sat. Level <= 0.465

NODE 1.1 => Sat. Level <= 0.115

NODE 1.2 => Sat. Level > 0.115

NODE 1.2.1 => Projects <= 2.5
Size of df121: 1738

Optimal Feature information for NODE 1.2.1 
 {'information_gain': 0.2228, 'weighted_entropy': 0.3106, 'original_entropy': 0.5334, 'feature_column': 'last_evaluation', 'split': 0.575}

NODE 1.2.2 => Projects > 2.5

Optimal Feature information for NODE 1.2.2 
 {'information_gain': 0.037, 'weighted_entropy': 0.3455, 'original_entropy': 0.3825, 'feature_column': 'number_project', 'split': 6.5}


{'information_gain': 0.037,
 'weighted_entropy': 0.3455,
 'original_entropy': 0.3825,
 'feature_column': 'number_project',
 'split': 6.5}